In [6]:
import pandas as pd
from data.data_preprocessing import DefaultENPreprocessor
from mb_generator import BalancedMiniBatchLoader

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer


tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

model = AutoModelForMaskedLM.from_pretrained("vinai/bertweet-base")
trainer = Trainer(model)
trainer.save_model(output_dir = "/home/mats/Documents/University/4_Semester/SDSProjectsSeminar/the-algorithm/trust_and_safety_models/toxicity/LOKAL_DIR/models")

2023-06-08 10:19:00.861909: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 10:19:00.953701: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 10:19:01.091213: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 10:19:01.091753: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 10:19:02.330763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

- Trainer.data_loader(self) --> data_frame_loader.py.DataFrameLoader

In [8]:
data = pd.read_csv("/home/mats/Documents/University/4_Semester/SDSProjectsSeminar/the-algorithm/trust_and_safety_models/toxicity/data/data.csv", index_col=0)
data.columns = ["label", "text"]

prepro = DefaultENPreprocessor()

df = prepro(df=data,
      label_column='label',
      class_weight=0.25,
      filter_low_agreements=False,
      num_classes=2)

#df.to_csv("check_preprocess.csv")

mb_loader = BalancedMiniBatchLoader(
      fold="time",
      seed=1337,
      #Should be 0.0 < perc_training_tox < 0.5
      perc_training_tox=0.25,
      mb_size=64,
      n_outer_splits=5,
      scope='TOX',
      # project=None,
      dual_head=False,
      sample_weights='class_weight',
      huggingface=True,
    )

#test data = result of split
#steps per epoch = simple integer
mini_batches, test_data, steps_per_epoch = mb_loader.simple_cv_load(df)

#data['tokens'] = data['text'].apply(lambda x: tokenizer.tokenize(x))

Got 56745 before cleaning
.... removing \n and replacing @mentions and URLs by placeholders. Emoji filtering is not done.
WARNING you are not removing tweets with media to train a BERT model.
.... deleting duplicates
Got 54079 after cleaning
Making a local copy of Bertweet-base model


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


5
.... 43264 examples, incl. 43.92% tox in train, 2 classes


/home/mats/.local/lib/python3.10/site-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


1187

In [7]:
from train import Trainer
test = Trainer(optimizer_name='Adam', 
               weight_decay=0.01, 
               learning_rate=0.01,
               mb_size=64,
               train_epochs=10)
test.train_full_model(data)

Making a local copy of Bertweet-base model


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


------- Experiment name:  en2023-05-2212:33:14_defaulttime_Adam_0.01_1_64_0.25_10_seed1337
------- Checking there is a GPU
Setting up random seed.
Loading en data
Going to train on everything but the test dataset
5
.... 45396 examples, incl. 42.56% tox in train, 2 classes


/home/mats/.local/lib/python3.10/site-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
                                                                   

Loaded callback for test, from_logits: True, labels ['']
.... using warm-up for 1207 steps
.... Adam w global clipnorm None
Loading model


NameError: name 'load' is not defined